### Imports

In [1]:
#Import all utility functions and required libraries
from AssociationTask import *

#Initializes display settings
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.cmap'] = 'gray'

2.7.0
There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


### Paths definition and sample selection
Define paths to images and predictions .csv and load them

In [2]:
#Enter path to images
PATH = "../data/"
#Prediction file path & datafile name
PREDICTION_PATH = ""
PREDICTION_NAME = "CEP63_final_test.csv"
#Output path
OUTPUT_PATH=""
#Name of ouput file
OUTPUT_NAME = "Association"

In [3]:
#Enter channel 0, 1, 2 as int
channel = 1
#Enter id of images as list of two int in [0,1,2]
id = [1,2]
#Marker 1, 2 or 3 for centrioles
mkr = 1


In [4]:
#Load images names and path
IMG_Paths = generatePaths(channel, id, path=PATH)
IMG_Names = generateNames(channel, id, path=PATH)

Ncl_Path = IMG_Paths[0][0]
Ncl_Name = IMG_Names[0]

Ctr_Path = IMG_Paths[0][mkr]
Ctr_Name = IMG_Names[mkr]

FileName = IMG_Paths[1]



In [5]:
#Load centrioles dataframe and centrioles prediction for the corresponding image
CTR_df = pd.read_csv(PREDICTION_PATH + PREDICTION_NAME)
Ctr_df = CTR_df[CTR_df['image_name'] == Ctr_Name]

In [6]:
#define image to be processed by nucleus detection algorithm
raw = plt.imread(Ncl_Path, cv.IMREAD_UNCHANGED)
raw = raw.astype(float)

### Prediction Block
Operates prediction of Nuclei detection and retrieve results for the association

In [7]:
# Compute gradients in x and y directions using the Sobel mask
grad_x = cv.Sobel(raw,ddepth=-1,dx=1,dy=0,ksize=31)
grad_y = cv.Sobel(raw,ddepth=-1,dx=0,dy=1,ksize=31)
grad_mag = np.sqrt(np.power(grad_x,2) + np.power(grad_y,2))
grad_scaled = grad_mag/np.max(grad_mag)*np.max(raw)

# Scale
images = [grad_scaled, grad_scaled+raw, raw]
resized = []

for image in images:
    image = cv.resize(image, (256,256))
    resized.append(image)
        
# Run Stardist
stack = []
for img in resized:
    labels, infos = model.predict_instances(normalize(img))
    stack.append(labels)
    points = infos['points']
    prob = infos['prob']
        
attention_mask = np.sum(np.array(stack),axis = 0)
blobs = measure.label(attention_mask>1)
    
# Add border
add = 50

scalled_gt = cv.copyMakeBorder(resized[-1], add, add, add, add, cv.BORDER_CONSTANT, value=[0, 0, 0])
mask = cv.copyMakeBorder(blobs, add, add, add, add, cv.BORDER_CONSTANT, value=[0, 0, 0])
output = cv.copyMakeBorder(labels, add, add, add, add, cv.BORDER_CONSTANT, value=[0, 0, 0])
    
# define size of data
size = 64
e = int(size/2)
    
all_ = []
for x,y,p in zip(points[:,0], points[:,1], prob[:]):
        
    # Compensate border 
    x += add
    y += add
        
    nuc = scalled_gt[x-e:x+e,y-e:y+e]
    crop = mask[x-e:x+e,y-e:y+e]
    out = output[x-e:x+e,y-e:y+e]
        
    nuc_mag = crop[32,32]
    out_mag = out[32,32]
        
    filtered = nuc*((crop == nuc_mag)+0.25*(crop!=nuc_mag).astype(int))
    out = out*(out==out_mag)
    
    filtered = (filtered-filtered.mean())/filtered.std()
    out = (out-out.mean())/out.std()
    
    tmp = np.array([filtered,out])
    all_.append(tmp)
    
input_ = torch.from_numpy(np.array(all_)).float()
input_.shape

output = model1(input_)
value,idx = output.max(1)
value,idx

(tensor([0.9953, 0.3997, 0.9921, 0.9964, 0.9459, 0.9905, 0.9997, 0.9396, 1.0000,
         0.9997, 1.0000, 0.9999, 0.9947, 1.0000, 0.9956, 0.9914, 1.0000, 0.9564,
         1.0000, 0.9951, 0.9994, 0.8564, 0.9996, 1.0000, 0.9999, 0.9980, 0.8617,
         1.0000, 0.9735, 0.8339, 0.8480, 0.9553, 0.9935],
        grad_fn=<MaxBackward0>),
 tensor([1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 2, 1, 1, 0, 0]))

### Association Process

In [8]:
#Assignment process
label_list = np.unique(labels)
_, _, _, CtrDic = AssingCtr(Ctr_df, labels, infos)

### .csv file creation

In [9]:
#Create ouput dataframe and add arrays to fill
Output_df = Ctr_df[['image_name', 'x', 'y', 'score']].rename(columns={'x':'ctr_x', 'y':'ctr_y', 'score':'ctr_score'})
nucleus_id = np.full([len(Ctr_df.x), 1], -1)
ncl_x = np.full([len(Ctr_df.x),1], -1)
ncl_y = np.full([len(Ctr_df.x),1], -1)
Sncl_score = np.zeros([len(Ctr_df.x),1])
ncl_class = np.full([len(Ctr_df.x), 1], -1)
class_conf = np.zeros([len(Ctr_df.x), 1])

In [10]:
for i, (x, y) in enumerate(zip(Ctr_df.x, Ctr_df.y)):
    for k, v in CtrDic.items():
        for ctr in v:
            if ctr[0] == x and ctr[1] == y:
                nucleus_id[i] = k
                ncl_x[i] = infos["points"][k-1][1]*8
                ncl_y[i] = infos["points"][k-1][0]*8
                Sncl_score[i] = infos["prob"][k-1]
                ncl_class[i] = idx[k-1]
                class_conf[i] = value[k-1].item()

Output_df['nucleus_id'] = nucleus_id
Output_df['ncl_x'] = ncl_x
Output_df['ncl_y'] = ncl_y
Output_df['Stardist_ncl_score'] = Sncl_score
Output_df['nucleus_class'] = ncl_class
Output_df['class_confidence'] = class_conf

In [11]:
Output_df.to_csv(OUTPUT_PATH+OUTPUT_NAME+".csv", index=False)